In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# ========== python ==========
import os
from logging import Logger
from pathlib import Path
import gc
# noinspection PyUnresolvedReferences
from typing import List, Dict, Tuple, Optional, Union, Callable, Final, Literal, get_args
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
PROJECT_DIR = Path().resolve().parents[0]
os.chdir(PROJECT_DIR)
SEED: Final = 42

In [2]:
from utils.setup import load_param, easy_logger, get_device
from utils.torch import load_model, torch_fix_seed
from torch.utils.tensorboard import SummaryWriter
from ignite.handlers import Checkpoint
from run_for_KGC import main_function
logger = easy_logger('log/test.log')

In [3]:
args = load_param('saved_models/kgc/all100/02/param.pkl')
device = get_device(device_name='cpu', logger=logger)
# args.pre_train = True
args.logger = logger
args.device = device
args.batch_size = 1
args.no_use_pe = False
args.max_len=512
args.tensorboard_dir=None
args.only_load_trainer_evaluator=True
args.pre_train=False
del args.optuna_file, args.device_name, args.pid
del args.study_name, args.n_trials
logger.info(args)

2022-12-06 23:24:52 - INFO - 1747899880.py - 14 - Namespace(notebook=False, train_anyway=False, logfile='saved_models/kgc/all100/02/log.log', param_file='saved_models/kgc/all100/02/param.pkl', tensorboard_dir=None, checkpoint_dir='saved_models/kgc/all100/02/checkpoint/', model_path='saved_models/kgc/all100/02/model.pth', model_version='02', resume_from_checkpoint=False, resume_from_last_point=False, only_load_trainer_evaluator=True, resume_checkpoint_path=None, console_level='info', pre_train=False, train_valid_test=False, only_train=True, use_for_challenge100=False, use_for_challenge075=False, use_title=None, do_optuna=False, story_special_num=5, relation_special_num=5, entity_special_num=5, padding_token_e=0, cls_token_e=1, mask_token_e=2, sep_token_e=3, bos_token_e=4, padding_token_r=0, cls_token_r=1, mask_token_r=2, sep_token_r=3, bos_token_r=4, padding_token_s=0, cls_token_s=1, mask_token_s=2, sep_token_s=3, bos_token_s=4, embedding_dim=128, separate_head_and_tail=False, batch_siz

In [4]:
torch_fix_seed(seed=SEED)
model, data_dict = main_function(args, logger=logger)
encoder_last_layer = model.transformer.layers[-1]
model.eval()
dataset_train, dataset_valid, dataset_test = data_dict['datasets']
triple = dataset_train.triple
data_helper = data_dict['data_helper']
evaluator = data_dict['train_items']['evaluator']
load_model(model, args.model_path, device)

2022-12-06 23:24:52 - INFO - run_for_KGC.py - 723 - ----- make datahelper start. -----
2022-12-06 23:24:52 - INFO - data_helper.py - 438 - ==========Show DataHelper==========
2022-12-06 23:24:52 - INFO - data_helper.py - 439 - ==========
2022-12-06 23:24:52 - INFO - data_helper.py - 205 - ==========Show MyRawData==========
2022-12-06 23:24:52 - INFO - data_helper.py - 206 - entity_length: 7857
2022-12-06 23:24:52 - INFO - data_helper.py - 207 - relation_length: 63
2022-12-06 23:24:52 - INFO - data_helper.py - 210 - ==========Show MyRawData==========
2022-12-06 23:24:52 - INFO - data_helper.py - 441 - ==========
2022-12-06 23:24:52 - INFO - data_helper.py - 443 - entity_special_num: 5
2022-12-06 23:24:52 - INFO - data_helper.py - 444 - relation_special_num: 5
2022-12-06 23:24:52 - INFO - data_helper.py - 445 - ==========Show DataHelper==========
2022-12-06 23:24:52 - INFO - run_for_KGC.py - 725 - ----- make datahelper complete. -----
2022-12-06 23:24:52 - INFO - run_for_KGC.py - 728 - -

KgStoryTransformer02(
  (emb_entity): Embedding(7862, 128, padding_idx=0)
  (emb_relation): Embedding(68, 128, padding_idx=0)
  (pe): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1028, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=1028, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
        (activation): GELU(approximate=none)
      )
      (1): TransformerEncoderLayer(
        (self_attn): 

In [5]:
# dataset_train, dataset_valid, dataset_test

In [6]:
# evaluator.run(data_helper.valid_dataloader)
# metrics = evaluator.state.metrics
# metrics

In [7]:
entities = data_helper.processed_entities
d_e = {e: i for i, e in enumerate(entities)}
relations = data_helper.processed_relations
d_r = {r: i for i, r in enumerate(relations)}
triple_df = pd.DataFrame([(entities[_t[0]], relations[_t[1]], entities[_t[2]]) for _t in triple], columns=['head', 'relation', 'tail']) 
# story_entities = triple_df
# triple_df

In [8]:
def extract(model, target, inputs):
    feature = None

    def forward_hook(module, inputs, _):
        # 順伝搬の出力を features というグローバル変数に記録する
        global features
        # 1. detach でグラフから切り離す。
        # 2. clone() でテンソルを複製する。モデルのレイヤーで ReLU(inplace=True) のように
        #    inplace で行う層があると、値がその後のレイヤーで書き換えられてまい、
        #    指定した層の出力が取得できない可能性があるため、clone() が必要。
        x, _, _ = inputs
        outputs = module.forward(x, x, x, need_weights=True)[1]
        features = outputs.detach().clone()

    # コールバック関数を登録する。
    handle = target.register_forward_hook(forward_hook)

    # 推論する
    model.eval()
    model(inputs, torch.LongTensor([[]]), torch.LongTensor([[]]), torch.LongTensor([[]]))

    # コールバック関数を解除する。
    handle.remove()

    return features

In [9]:
def heatmap_(index):
    input_ = dataset[index]
    # display([(i, entities[h], relations[r], entities[t]) for i, (h, r, t) in enumerate(input_)])
    features = extract(model, encoder_last_layer.self_attn, dataset[0][None, :])[0]
    df_atten = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_)])
    df_atten.columns=['head', 'relation', 'tail',] + [f'atten_from{i}' for i in range(len(df_atten.columns)-3)]
    display(df_atten)
    # ヒートマップの作成
    # sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
    for i in range(len(df_atten)):
        display(i, df_atten.iloc[i,:3].tolist())
        display(df_atten.sort_values(f'atten_from{i}', ascending=False).iloc[:10,:3])
        print("----------")
    plt.show()
# heatmap_(300)

In [10]:
def make_heatmap2_(input_):
    assert len(input_) == 1
    features = extract(model, encoder_last_layer.self_attn, input_)[0]
    df_atten = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_[0])])
    df_atten.columns=['head', 'relation', 'tail',] + [f'atten_from{i}' for i in range(len(df_atten.columns)-3)]
    return df_atten

In [11]:
print(f"{model.weight_head.data=}")
print(f"{model.weight_relation.data=}")
print(f"{model.weight_tail.data=}")
story_entities = triple_df['head'].tolist()
display(story_entities)

model.weight_head.data=tensor(1.3410)
model.weight_relation.data=tensor(0.3783)
model.weight_tail.data=tensor(0.5853)


['<bos_e>',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 '<bos_e>',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 '<bos_e>',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 '<bos_e>',
 'AbbeyGrange:004',
 'AbbeyGrange:004',
 'AbbeyGrange:004',
 '<bos_e>',
 'AbbeyGrange:005',
 'AbbeyGrange:005',
 'AbbeyGrange:005',
 'AbbeyGrange:005',
 '<bos_e>',
 'AbbeyGrange:006',
 'AbbeyGrange:006',
 '<bos_e>',
 'AbbeyGrange:007',
 'AbbeyGrange:007',
 'AbbeyGrange:007',
 'AbbeyGrange:007',
 '<bos_e>',
 'AbbeyGrange:008',
 'AbbeyGrange:008',
 'AbbeyGrange:008',
 'AbbeyGrange:008',
 'AbbeyGrange:008',
 '<bos_e>',
 'AbbeyGrange:009',
 'AbbeyGrange:009',
 '<bos_e>',
 'AbbeyGrange:009a',
 'AbbeyGrange:009a'

In [61]:
def make_ranking(_from, _to, _predicate, _whom, _subject, _why, _what, _where):
    if not (_from is None and _to is None):
        _start_index = story_entities.index(_from)-1 
        _end_index = len(story_entities) - story_entities[::-1].index(_to)
    else: 
        _start_index = 0
        _end_index = 0
    question_ = torch.tensor(
        [
            [d_e['<bos_e>'],d_r['<bos_r>'],d_e['<bos_e>']],
            [d_e['<mask_e>'],d_r['kgc:hasPredicate'],d_e[_predicate]],
            [d_e['<mask_e>'],d_r['kgc:whom'],d_e[_whom]],
            [d_e['<mask_e>'],d_r['kgc:subject'],d_e[_subject]],
            [d_e['<mask_e>'],d_r['kgc:why'],d_e[_why]],
            [d_e['<mask_e>'],d_r['kgc:what'],d_e[_what]],
            [d_e['<mask_e>'],d_r['kgc:where'],d_e[_where]],
        ]
    )
    mask_ = torch.tensor(
        [
            [False, False, False],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
        ]
    )
    questions = torch.cat([triple[_start_index: _end_index], question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros(_end_index-_start_index,3), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)
    tmp = []
    with torch.no_grad():
        _, (story_pred, relation_pred, entity_pred) = model(questions, masks[:,0], masks[:,1], masks[:,2])
        sorted_ = torch.argsort(entity_pred.to('cpu'), dim=1, descending=True)
        for i in range(sorted_.shape[1]):
            ans_= sorted_[:, i]
            predicate_, whom_, subj_, why_, what_, where_ = ans_
            tmp.append([entities[predicate_], entities[whom_], entities[subj_], entities[why_], entities[what_], entities[where_]])
    df_ranking = pd.DataFrame(tmp, columns=['predicate', 'whom', 'subject', 'why', 'what', 'where'])
    df_attension = make_heatmap2_(questions)
    return df_ranking, df_attension

In [62]:
MASK_E = '<mask_e>'
KILL = 'word.predicate:kill'
TAKE = 'word.predicate:take'
BRING = 'word.predicate:bring'
DIE = 'word.predicate:die'
HIDE = 'word.predicate:hide'

### まだらの紐
Who killed Julia? (criminal & explanation)
被害者: Julia
犯人: Roylott
犯行に用いたもの: snake
犯行動機: 母の相続財産を独占したい

In [63]:
victim = 'SpeckledBand:Julia'
killer = 'SpeckledBand:Roylott'
df_ranking_SpeckledBand, df_attension_SpeckledBand = make_ranking(
    None, None, KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_SpeckledBand.index[df_ranking_SpeckledBand['subject']==killer].tolist())
display(df_ranking_SpeckledBand.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_SpeckledBand)
for i in range(len_):
    display(i, df_attension_SpeckledBand.iloc[i,:3].tolist())
    display(df_attension_SpeckledBand.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

SpeckledBand:Roylott, ranking [259]


,predicate,whom,subject,why,what,where
0,word.predicate:avoid,word.predicate:say,SilverBlaze:Inspector_Gregory,word.predicate:avoid,word.predicate:say,word.predicate:wander
1,word.predicate:kill,SilverBlaze:Night_of_the_incident,SilverBlaze:Fitzroy_Simpson,word.predicate:wander,word.predicate:wander,SilverBlaze:wilderness
2,word.predicate:testify,SilverBlaze:Inspector_Gregory,SilverBlaze:Silver_Blaze,word.predicate:say,SilverBlaze:Silver_Blaze,word.predicate:say
3,word.predicate:bring,SilverBlaze:Silver_Blaze,SilverBlaze:John_Straker,SilverBlaze:Night_of_the_incident,word.predicate:avoid,word.predicate:go
4,word.predicate:bet,word.predicate:bring,SilverBlaze:A_group_of_Gypsy,SilverBlaze:In_combat,SilverBlaze:camp,SilverBlaze:Silas_Brown_stables
5,word.predicate:suspect,word.predicate:gain,word.predicate:say,<bos_e>,word.predicate:testify,word.predicate:hold
6,word.predicate:wander,word.predicate:wander,word.predicate:wander,SilverBlaze:Silver_Blaze,<bos_e>,SilverBlaze:King_s_Pyland
7,word.predicate:contain,word.predicate:meet,SilverBlaze:knife,word.predicate:canParticipate,SilverBlaze:A_group_of_Gypsy,word.predicate:sleep
8,word.predicate:notHave,SilverBlaze:John_Straker,SilverBlaze:Night_of_the_incident,SilverBlaze:Insufficient_reasons,word.predicate:hold,word.predicate:notHave
9,word.predicate:give,ResidentPatient:Blessington,word.predicate:avoid,SilverBlaze:192,SilverBlaze:knife,word.predicate:testify


0

['<bos_e>', '<bos_r>', '<bos_e>']

,head,relation,tail,atten_from0
2,<mask_e>,kgc:whom,SpeckledBand:Julia,0.255900
0,<bos_e>,<bos_r>,<bos_e>,0.216392
5,<mask_e>,kgc:what,<mask_e>,0.144973
4,<mask_e>,kgc:why,<mask_e>,0.128166
6,<mask_e>,kgc:where,<mask_e>,0.108208
3,<mask_e>,kgc:subject,<mask_e>,0.093675
1,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.052686


...

----------


6

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from6
2,<mask_e>,kgc:whom,SpeckledBand:Julia,0.340451
0,<bos_e>,<bos_r>,<bos_e>,0.316299
5,<mask_e>,kgc:what,<mask_e>,0.085117
6,<mask_e>,kgc:where,<mask_e>,0.083060
4,<mask_e>,kgc:why,<mask_e>,0.071996
3,<mask_e>,kgc:subject,<mask_e>,0.061465
1,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.041613


----------


In [14]:
victim = 'SpeckledBand:Julia'
df_ranking_SpeckledBand, df_attension_SpeckledBand = make_ranking(
    'SpeckledBand:330', 'SpeckledBand:400', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_SpeckledBand.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_SpeckledBand)
for i in range(len_-20, len_):
    display(i, df_attension_SpeckledBand.iloc[i,:3].tolist())
    display(df_attension_SpeckledBand.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:cannotFind,SpeckledBand:snake,SpeckledBand:snake,SpeckledBand:1883-04-01T23,SpeckledBand:snake,SpeckledBand:safe
1,word.predicate:kill,AllTitle:Holmes,AllTitle:Holmes,SpeckledBand:snake,SpeckledBand:sound_of_snake,AbbeyGrange:train
2,word.predicate:return,SpeckledBand:Roylott,SpeckledBand:Roylott,SpeckledBand:sound_of_snake,SpeckledBand:metallic_sound,SpeckledBand:snake
3,word.predicate:bite,SpeckledBand:Helen,SpeckledBand:Helen,DateTime:1883-04-01T23:00:00,SpeckledBand:Roylott,SpeckledBand:Roylott_s_bedroom
4,word.predicate:hit,AllTitle:Watson,SpeckledBand:Safe,word.predicate:bite,AllTitle:Holmes,SpeckledBand:mansion_of_Roylott
5,word.predicate:hear,word.predicate:angry,SpeckledBand:whip,SpeckledBand:VentilationHole,AbbeyGrange:letter,AbbeyGrange:Abbey_Grange
6,word.predicate:goDown,DateTime:1883-04-01T23:00:00,SpeckledBand:sound_of_snake,word.predicate:angry,SpeckledBand:safe,SpeckledBand:Roylott
7,word.predicate:reach,word.predicate:return,SpeckledBand:1883-04-01T23,word.predicate:return,SpeckledBand:Helen,SpeckledBand:sound_of_snake
8,word.predicate:putInto,SpeckledBand:VentilationHole,SpeckledBand:safe,<bos_e>,<bos_e>,SpeckledBand:India
9,word.predicate:go,<bos_e>,SpeckledBand:milk,AllTitle:Holmes,AbbeyGrange:006,SpeckledBand:1883-04-01T23


381

['SpeckledBand:398', 'kgc:time', 'DateTime:1883-04-01T23:00:00']

,head,relation,tail,atten_from381
375,SpeckledBand:397,kgc:time,DateTime:1883-04-01T23:00:00,0.042594
390,SpeckledBand:400,kgc:hasPredicate,word.predicate:bite,0.033843
381,SpeckledBand:398,kgc:time,DateTime:1883-04-01T23:00:00,0.030526
364,SpeckledBand:395,kgc:time,DateTime:1883-04-01T23:00:00,0.028682
391,SpeckledBand:400,kgc:what,SpeckledBand:Roylott,0.026474
393,SpeckledBand:400,kgc:time,DateTime:1883-04-01T23:00:00,0.023171
370,SpeckledBand:396,kgc:time,DateTime:1883-04-01T23:00:00,0.022709
378,SpeckledBand:398,kgc:hasPredicate,word.predicate:return,0.021576
379,SpeckledBand:398,kgc:to,SpeckledBand:VentilationHole,0.021039
392,SpeckledBand:400,kgc:when,SpeckledBand:1883-04-01T23,0.021015


----------


...

----------


400

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from400
393,SpeckledBand:400,kgc:time,DateTime:1883-04-01T23:00:00,0.062672
381,SpeckledBand:398,kgc:time,DateTime:1883-04-01T23:00:00,0.058865
375,SpeckledBand:397,kgc:time,DateTime:1883-04-01T23:00:00,0.045056
390,SpeckledBand:400,kgc:hasPredicate,word.predicate:bite,0.042465
387,SpeckledBand:399,kgc:time,DateTime:1883-04-01T23:00:00,0.039452
399,<mask_e>,kgc:what,<mask_e>,0.037132
391,SpeckledBand:400,kgc:what,SpeckledBand:Roylott,0.030713
364,SpeckledBand:395,kgc:time,DateTime:1883-04-01T23:00:00,0.027462
392,SpeckledBand:400,kgc:when,SpeckledBand:1883-04-01T23,0.026382
370,SpeckledBand:396,kgc:time,DateTime:1883-04-01T23:00:00,0.025910


----------


### 悪魔の足跡１
Who killed the victims? (criminal & explanation)
被害者: Brenda
犯人: Mortimer
犯行動機: 財産

In [64]:
victim = 'DevilsFoot:Brenda'
killer = 'DevilsFoot:Mortimer'
df_ranking_DevilsFoot1, df_attension_DevilsFoot1 = make_ranking(
    None, None, KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_DevilsFoot1.index[df_ranking_DevilsFoot1['subject'] == killer].tolist()[0])
display(df_ranking_DevilsFoot1.iloc[:20,:])

# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_DevilsFoot1)
for i in range(len_):
    display(i, df_attension_DevilsFoot1.iloc[i,:3].tolist())
    display(df_attension_DevilsFoot1.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

DevilsFoot:Mortimer, ranking 50


,predicate,whom,subject,why,what,where
0,word.predicate:testify,word.predicate:hold,SilverBlaze:Fitzroy_Simpson,SilverBlaze:In_combat,SilverBlaze:Silver_Blaze,SilverBlaze:wilderness
1,word.predicate:notHave,SilverBlaze:John_Straker,SilverBlaze:John_Straker,SilverBlaze:Fitzroy_Simpson,SilverBlaze:Head_of_John_Straker,word.predicate:hold
2,word.predicate:suspect,word.predicate:become,SilverBlaze:Silver_Blaze,SilverBlaze:192,SilverBlaze:In_combat,word.predicate:wander
3,word.predicate:hold,DevilsFoot:Brenda,SilverBlaze:Inspector_Gregory,SilverBlaze:191,SilverBlaze:knife,SilverBlaze:Silas_Brown_stables
4,word.predicate:contain,SilverBlaze:Night_of_the_incident,SilverBlaze:knife,SilverBlaze:188,SilverBlaze:Fitzroy_Simpson,SilverBlaze:Open_air
5,word.predicate:wander,word.predicate:meet,SilverBlaze:A_group_of_Gypsy,SilverBlaze:Silver_Blaze,SilverBlaze:186,SilverBlaze:knife
6,word.predicate:self-harm,word.predicate:manage,SilverBlaze:Corpse_of_John_Straker,SilverBlaze:190,SilverBlaze:candle,word.predicate:go
7,word.predicate:kill,SilverBlaze:1896-04-02T00:00,SilverBlaze:wilderness,SilverBlaze:Head_of_John_Straker,SilverBlaze:Thick_stick,SilverBlaze:Curry_dish_of_Ned_Hunter
8,word.predicate:avoid,SilverBlaze:Silver_Blaze,SilverBlaze:Wife_of_John_Straker,SilverBlaze:189,SilverBlaze:tie,SilverBlaze:Fitzroy_Simpson
9,word.predicate:become,SilverBlaze:Fitzroy_Simpson,SilverBlaze:Horse,SilverBlaze:273,word.predicate:hold,SilverBlaze:Silver_Blaze


0

['<bos_e>', '<bos_r>', '<bos_e>']

,head,relation,tail,atten_from0
0,<bos_e>,<bos_r>,<bos_e>,0.233659
5,<mask_e>,kgc:what,<mask_e>,0.170707
2,<mask_e>,kgc:whom,DevilsFoot:Brenda,0.163049
4,<mask_e>,kgc:why,<mask_e>,0.132533
6,<mask_e>,kgc:where,<mask_e>,0.117240
3,<mask_e>,kgc:subject,<mask_e>,0.101302
1,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.081510


...

----------


6

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from6
2,<mask_e>,kgc:whom,DevilsFoot:Brenda,0.289802
0,<bos_e>,<bos_r>,<bos_e>,0.266778
3,<mask_e>,kgc:subject,<mask_e>,0.110762
5,<mask_e>,kgc:what,<mask_e>,0.097785
6,<mask_e>,kgc:where,<mask_e>,0.092083
4,<mask_e>,kgc:why,<mask_e>,0.077956
1,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.064834


----------


In [15]:
victim = 'DevilsFoot:Brenda'
df_ranking_DevilsFoot1, df_attension_DevilsFoot1 = make_ranking(
    'DevilsFoot:420', 'DevilsFoot:488', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_DevilsFoot1.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_DevilsFoot1)
for i in range(len_-20, len_):
    display(i, df_attension_DevilsFoot1.iloc[i,:3].tolist())
    display(df_attension_DevilsFoot1.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:kill,DevilsFoot:Brenda,DevilsFoot:Sterndale,DevilsFoot:money,DevilsFoot:Powder_medicine,DevilsFoot:garden_of_Cottage_of_Sterndale
1,word.predicate:check,DevilsFoot:Sterndale,DevilsFoot:Mortimer,DevilsFoot:Powder_medicine,DevilsFoot:gun,DevilsFoot:Africa
2,word.predicate:wander,DevilsFoot:Owen,DevilsFoot:Powder_medicine,DevilsFoot:Another_interpretation,DevilsFoot:Another_interpretation,DevilsFoot:garden_of_pastoral_hall
3,word.predicate:fire,AllTitle:Holmes,AbbeyGrange:Criminal,AbbeyGrange:221,DevilsFoot:Sterndale,DevilsFoot:Outside_of_window
4,word.predicate:stand,DevilsFoot:Mortimer,AllTitle:Holmes,DevilsFoot:laws_of_United_Kingdom,DevilsFoot:fire,DevilsFoot:Europe
5,word.predicate:die,DevilsFoot:George,DevilsFoot:Another_interpretation,DevilsFoot:Five_minutes_later,DevilsFoot:Information,DevilsFoot:paper_packet
6,word.predicate:put,DevilsFoot:Brother,DevilsFoot:European_chemistry,DevilsFoot:Sterndale,DevilsFoot:460,DevilsFoot:Sterndale
7,word.predicate:notExist,<bos_e>,word.predicate:notExist,DevilsFoot:gravel,DevilsFoot:gravel,DevilsFoot:Sea
8,word.predicate:show,DevilsFoot:Exclusive_manager_of_shared_property,<bos_e>,AbbeyGrange:219,DevilsFoot:459,DevilsFoot:lawn
9,word.predicate:beUsed,word.predicate:meet,DevilsFoot:gravel,AbbeyGrange:200,DevilsFoot:448,word.predicate:wander


304

['DevilsFoot:485', 'kgc:what', 'DevilsFoot:gun']

,head,relation,tail,atten_from304
319,<mask_e>,kgc:whom,DevilsFoot:Brenda,0.047456
304,DevilsFoot:485,kgc:what,DevilsFoot:gun,0.047002
303,DevilsFoot:485,kgc:hasPredicate,word.predicate:hold,0.034903
318,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.027333
256,DevilsFoot:473,kgc:what,DevilsFoot:483,0.023130
282,DevilsFoot:480,kgc:hasPredicate,word.predicate:shoot,0.021358
252,DevilsFoot:473,kgc:what,DevilsFoot:479,0.020132
322,<mask_e>,kgc:what,<mask_e>,0.019683
257,DevilsFoot:473,kgc:what,DevilsFoot:484,0.019628
258,DevilsFoot:473,kgc:what,DevilsFoot:485,0.018274


----------


...

----------


323

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from323
319,<mask_e>,kgc:whom,DevilsFoot:Brenda,0.112661
310,DevilsFoot:487,kgc:subject,DevilsFoot:Starting_point_of_investigation,0.044756
322,<mask_e>,kgc:what,<mask_e>,0.035589
316,DevilsFoot:488,kgc:where,DevilsFoot:garden_of_pastoral_hall,0.030131
318,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.027820
323,<mask_e>,kgc:where,<mask_e>,0.027743
306,DevilsFoot:486,kgc:subject,DevilsFoot:Mortimer,0.026872
320,<mask_e>,kgc:subject,<mask_e>,0.026284
254,DevilsFoot:473,kgc:what,DevilsFoot:481,0.025006
314,DevilsFoot:488,kgc:subject,DevilsFoot:gravel,0.022644


----------


### 悪魔の足跡2
Who killed the victims? (criminal & explanation)
被害者: Mortimer
犯人: 
犯行動機: 恋人の敵

In [65]:
victim = 'DevilsFoot:Mortimer'
killer = 'DevilsFoot:Sterndale'
df_ranking_DevilsFoot1, df_attension_DevilsFoot1 = make_ranking(
    None, None, KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_DevilsFoot1.index[df_ranking_DevilsFoot1['subject'] == killer].tolist()[0])
display(df_ranking_DevilsFoot1.iloc[:20,:])

# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_DevilsFoot1)
for i in range(len_):
    display(i, df_attension_DevilsFoot1.iloc[i,:3].tolist())
    display(df_attension_DevilsFoot1.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

DevilsFoot:Sterndale, ranking 7


,predicate,whom,subject,why,what,where
0,word.predicate:wander,word.predicate:hold,word.predicate:wander,DevilsFoot:money,SilverBlaze:knife,word.predicate:hold
1,word.predicate:hold,word.predicate:say,SilverBlaze:knife,word.predicate:wander,word.predicate:hold,word.predicate:wander
2,word.predicate:notHave,word.predicate:go,AllTitle:Holmes,SilverBlaze:272,<bos_e>,word.predicate:go
3,word.predicate:suspect,word.predicate:think,<bos_e>,<bos_e>,word.predicate:say,SilverBlaze:Silas_Brown_stables
4,word.predicate:testify,word.predicate:meet,AllTitle:Watson,SilverBlaze:278,word.predicate:wander,<bos_e>
5,word.predicate:kill,word.predicate:become,word.predicate:say,SilverBlaze:191,SilverBlaze:paper,word.predicate:say
6,word.predicate:go,word.predicate:handOver,SilverBlaze:paper,SilverBlaze:273,SilverBlaze:Letter_of_instructions_from_Colone...,SilverBlaze:The_bottom_of_the_depression
7,word.predicate:handOver,<bos_e>,DevilsFoot:Sterndale,SilverBlaze:270,SilverBlaze:Three_pieces_of_receipt_already_in...,SilverBlaze:knife
8,word.predicate:say,DevilsFoot:Mortimer,DevilsFoot:Mortimer,SilverBlaze:271,SilverBlaze:Place_of_Silver_Blaze,SilverBlaze:The_following_horse_racing
9,word.predicate:become,word.predicate:find,SilverBlaze:Inspector_Gregory,word.predicate:consult,SilverBlaze:Silver_Blaze,SilverBlaze:King_s_Pyland


0

['<bos_e>', '<bos_r>', '<bos_e>']

,head,relation,tail,atten_from0
2,<mask_e>,kgc:whom,DevilsFoot:Mortimer,0.184417
5,<mask_e>,kgc:what,<mask_e>,0.171995
0,<bos_e>,<bos_r>,<bos_e>,0.163528
6,<mask_e>,kgc:where,<mask_e>,0.144002
4,<mask_e>,kgc:why,<mask_e>,0.130367
3,<mask_e>,kgc:subject,<mask_e>,0.104645
1,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.101045


...

----------


6

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from6
2,<mask_e>,kgc:whom,DevilsFoot:Mortimer,0.322025
0,<bos_e>,<bos_r>,<bos_e>,0.203524
6,<mask_e>,kgc:where,<mask_e>,0.114817
5,<mask_e>,kgc:what,<mask_e>,0.113813
3,<mask_e>,kgc:subject,<mask_e>,0.093295
4,<mask_e>,kgc:why,<mask_e>,0.084803
1,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.067722


----------


In [16]:
victim = 'DevilsFoot:Mortimer'
df_ranking_DevilsFoot2, df_attension_DevilsFoot2 = make_ranking(
    'DevilsFoot:420', 'DevilsFoot:488', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_DevilsFoot2.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_DevilsFoot2)
for i in range(len_-20, len_):
    display(i, df_attension_DevilsFoot2.iloc[i,:3].tolist())
    display(df_attension_DevilsFoot2.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:kill,DevilsFoot:Mortimer,DevilsFoot:Sterndale,DevilsFoot:money,DevilsFoot:Powder_medicine,DevilsFoot:Africa
1,word.predicate:check,DevilsFoot:Sterndale,DevilsFoot:Mortimer,DevilsFoot:Powder_medicine,DevilsFoot:gun,DevilsFoot:garden_of_Cottage_of_Sterndale
2,word.predicate:wander,AllTitle:Holmes,DevilsFoot:Powder_medicine,DevilsFoot:Another_interpretation,DevilsFoot:Information,DevilsFoot:Outside_of_window
3,word.predicate:stand,DevilsFoot:Owen,AllTitle:Holmes,AbbeyGrange:221,DevilsFoot:Another_interpretation,DevilsFoot:garden_of_pastoral_hall
4,word.predicate:fire,DevilsFoot:George,AbbeyGrange:Criminal,DevilsFoot:laws_of_United_Kingdom,DevilsFoot:Sterndale,DevilsFoot:Sterndale
5,word.predicate:die,DevilsFoot:Brother,DevilsFoot:Another_interpretation,DevilsFoot:Five_minutes_later,DevilsFoot:fire,DevilsFoot:Europe
6,word.predicate:put,DevilsFoot:Exclusive_manager_of_shared_property,DevilsFoot:European_chemistry,DevilsFoot:Sterndale,DevilsFoot:453,DevilsFoot:lawn
7,word.predicate:enter,DevilsFoot:Brenda,<bos_e>,DevilsFoot:gravel,DevilsFoot:448,DevilsFoot:Sea
8,word.predicate:become,word.predicate:meet,DevilsFoot:Magic_foot,DevilsFoot:Magic_foot,DevilsFoot:460,DevilsFoot:paper_packet
9,word.predicate:beUsed,DevilsFoot:neighbor,word.predicate:notExist,DevilsFoot:Chair,DevilsFoot:459,word.predicate:wander


304

['DevilsFoot:485', 'kgc:what', 'DevilsFoot:gun']

,head,relation,tail,atten_from304
319,<mask_e>,kgc:whom,DevilsFoot:Mortimer,0.046982
304,DevilsFoot:485,kgc:what,DevilsFoot:gun,0.046647
303,DevilsFoot:485,kgc:hasPredicate,word.predicate:hold,0.034422
318,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.026584
256,DevilsFoot:473,kgc:what,DevilsFoot:483,0.023880
282,DevilsFoot:480,kgc:hasPredicate,word.predicate:shoot,0.021326
252,DevilsFoot:473,kgc:what,DevilsFoot:479,0.020250
257,DevilsFoot:473,kgc:what,DevilsFoot:484,0.020152
322,<mask_e>,kgc:what,<mask_e>,0.018822
258,DevilsFoot:473,kgc:what,DevilsFoot:485,0.018718


----------


...

----------


323

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from323
319,<mask_e>,kgc:whom,DevilsFoot:Mortimer,0.127997
310,DevilsFoot:487,kgc:subject,DevilsFoot:Starting_point_of_investigation,0.043018
322,<mask_e>,kgc:what,<mask_e>,0.035138
316,DevilsFoot:488,kgc:where,DevilsFoot:garden_of_pastoral_hall,0.030385
323,<mask_e>,kgc:where,<mask_e>,0.028289
318,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.027090
320,<mask_e>,kgc:subject,<mask_e>,0.025424
254,DevilsFoot:473,kgc:what,DevilsFoot:481,0.025274
306,DevilsFoot:486,kgc:subject,DevilsFoot:Mortimer,0.023731
314,DevilsFoot:488,kgc:subject,DevilsFoot:gravel,0.022561


----------


### 背中の曲がった男: 
Why did Barclay die?
被害者: Barclay
犯人: 
犯行動機: 

In [66]:
victim = 'CrookedMan:Barclay'
killer = 'CrookedMan:apoplexia'
df_ranking_CrookedMan, df_attension_CrookedMan = make_ranking(
    None, None, DIE, MASK_E, victim, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_CrookedMan.index[df_ranking_CrookedMan['why'] == killer].tolist()[0])
display(df_ranking_CrookedMan.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_CrookedMan)
for i in range(len_-20, len_):
    display(i, df_attension_CrookedMan.iloc[i,:3].tolist())
    display(df_attension_CrookedMan.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

CrookedMan:apoplexia, ranking 611


,predicate,whom,subject,why,what,where
0,word.predicate:wander,SilverBlaze:camp,SilverBlaze:Fitzroy_Simpson,SilverBlaze:In_combat,SilverBlaze:Place_of_Silver_Blaze,SilverBlaze:Open_air
1,word.predicate:go,SilverBlaze:Silver_Blaze,SilverBlaze:A_group_of_Gypsy,SilverBlaze:192,SilverBlaze:Head_of_John_Straker,word.predicate:go
2,word.predicate:bring,<bos_e>,<bos_e>,<bos_e>,SilverBlaze:knife,ACaseOfIdentity:Dance_party
3,word.predicate:fight,SilverBlaze:Place_of_Silver_Blaze,CrookedMan:Barclay,SilverBlaze:270,<bos_e>,<bos_e>
4,word.predicate:notHave,word.predicate:go,SilverBlaze:rain,SilverBlaze:273,SilverBlaze:camp,word.predicate:wander
5,word.predicate:meet,SilverBlaze:186,SilverBlaze:Silver_Blaze,SilverBlaze:Head_of_John_Straker,SilverBlaze:186,SilverBlaze:wilderness
6,word.predicate:have,SilverBlaze:In_combat,SilverBlaze:camp,SilverBlaze:278,SilverBlaze:In_combat,SilverBlaze:knife
7,word.predicate:self-harm,word.predicate:meet,SilverBlaze:Place_of_Silver_Blaze,SilverBlaze:191,SilverBlaze:Thick_stick,SilverBlaze:Place_of_Silver_Blaze
8,word.predicate:hide,word.predicate:self-harm,SilverBlaze:knife,SilverBlaze:186,SilverBlaze:Silver_Blaze,SilverBlaze:King_s_Pyland
9,word.predicate:say,SilverBlaze:Head_of_John_Straker,SilverBlaze:Depression_in_the_direction_of_Map...,SilverBlaze:271,SilverBlaze:Duplicate_key_of_the_stables,SilverBlaze:Silas_Brown_stables


IndexError: single positional indexer is out-of-bounds

In [22]:
victim = 'CrookedMan:Barclay'
df_ranking_CrookedMan, df_attension_CrookedMan = make_ranking(
    'CrookedMan:300', 'CrookedMan:372', DIE, MASK_E, victim, MASK_E, MASK_E, MASK_E)

display(df_ranking_CrookedMan.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_CrookedMan)
for i in range(len_-20, len_):
    display(i, df_attension_CrookedMan.iloc[i,:3].tolist())
    display(df_attension_CrookedMan.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:die,AllTitle:Holmes,CrookedMan:Barclay,CrookedMan:apoplexia,CrookedMan:Davide,CrookedMan:episode_of_Uriah_and_Bathsheba
1,word.predicate:call,CrookedMan:Barclay,AllTitle:Holmes,CrookedMan:Davide,CrookedMan:same_kind_of_sin,CrookedMan:living_room
2,word.predicate:see,AbbeyGrange:Lady_Brackenstall,CrookedMan:Davide,word.predicate:die,CrookedMan:preface_of_Samuel,CrookedMan:Old_Testament
3,word.predicate:exist,CrookedMan:police,DevilsFoot:Mortimer,CrookedMan:371,CrookedMan:stick,CrookedMan:main_street
4,word.predicate:pale,CrookedMan:Davide,AllTitle:Watson,<bos_e>,CrookedMan:criminal,CrookedMan:curtain
5,word.predicate:notContradict,CrookedMan:Murphy,CrookedMan:Nancy,word.predicate:David,CrookedMan:postscript_of_Samuel,CrookedMan:box
6,word.predicate:help,AbbeyGrange:Sir_Eustace_Brackenstall,CrookedMan:sin_of_Davide,CrookedMan:373,CrookedMan:370,CrookedMan:lodge
7,word.predicate:return,CrookedMan:Nancy,AbbeyGrange:Jack_Croker,CrookedMan:363a,CrookedMan:Teddy,CrookedMan:back_of_head_of_Barclay
8,word.predicate:live,AllTitle:Watson,<bos_e>,CrookedMan:369,CrookedMan:beast,AllTitle:Holmes
9,word.predicate:arrive,CrookedMan:episode_of_Uriah_and_Bathsheba,CrookedMan:Henry,CrookedMan:372,CrookedMan:key_of_living_room,CrookedMan:Hudson_street


319

['CrookedMan:369', 'kgc:where', 'CrookedMan:Old_Testament']

,head,relation,tail,atten_from319
335,<mask_e>,kgc:subject,CrookedMan:Barclay,0.039753
330,CrookedMan:372,kgc:hasPredicate,word.predicate:exist,0.037209
333,<mask_e>,kgc:hasPredicate,word.predicate:die,0.032188
309,CrookedMan:368,kgc:subject,AllTitle:Holmes,0.027793
337,<mask_e>,kgc:what,<mask_e>,0.027121
329,CrookedMan:372,kgc:subject,CrookedMan:sin_of_Davide,0.026187
317,CrookedMan:369,kgc:subject,CrookedMan:Davide,0.023100
325,CrookedMan:371,kgc:subject,CrookedMan:Barclay,0.023084
334,<mask_e>,kgc:whom,<mask_e>,0.022878
319,CrookedMan:369,kgc:where,CrookedMan:Old_Testament,0.019280


----------


...

----------


338

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from338
335,<mask_e>,kgc:subject,CrookedMan:Barclay,0.058491
334,<mask_e>,kgc:whom,<mask_e>,0.043377
337,<mask_e>,kgc:what,<mask_e>,0.038459
336,<mask_e>,kgc:why,<mask_e>,0.030610
309,CrookedMan:368,kgc:subject,AllTitle:Holmes,0.030019
325,CrookedMan:371,kgc:subject,CrookedMan:Barclay,0.029632
331,CrookedMan:372,kgc:where,CrookedMan:episode_of_Uriah_and_Bathsheba,0.029020
338,<mask_e>,kgc:where,<mask_e>,0.027173
330,CrookedMan:372,kgc:hasPredicate,word.predicate:exist,0.025600
324,CrookedMan:371,kgc:subject,CrookedMan:Davide,0.024088


----------


### 僧坊荘園
Who killed Lord Blackenstall? (criminal & explanation)
被害者: Sir_Eustace_Brackenstall
犯人: 
犯行動機: 

In [69]:
victim = 'AbbeyGrange:Sir_Eustace_Brackenstall'
killer = 'AbbeyGrange:Jack_Croker'
df_ranking_AbbeyGrange, df_attension_AbbeyGrange = make_ranking(
    None, None, KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_AbbeyGrange.index[df_ranking_AbbeyGrange['subject'] == killer].tolist()[0])
display(df_ranking_AbbeyGrange.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_AbbeyGrange)
for i in range(len_):
    display(i, df_attension_AbbeyGrange.iloc[i,:3].tolist())
    display(df_attension_AbbeyGrange.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

AbbeyGrange:Jack_Croker, ranking 14


,predicate,whom,subject,why,what,where
0,word.predicate:testify,SilverBlaze:Night_of_the_incident,SilverBlaze:Fitzroy_Simpson,SilverBlaze:Fitzroy_Simpson,SilverBlaze:Thick_stick,SilverBlaze:wilderness
1,word.predicate:kill,SilverBlaze:Inspector_Gregory,SilverBlaze:Inspector_Gregory,SilverBlaze:Night_of_the_incident,SilverBlaze:Fitzroy_Simpson,word.predicate:wander
2,word.predicate:wander,word.predicate:hit,SilverBlaze:John_Straker,<bos_e>,SilverBlaze:Silver_Blaze,SilverBlaze:Wessex_Cup_game
3,word.predicate:notHave,AbbeyGrange:Lady_Brackenstall,SilverBlaze:wilderness,SilverBlaze:wilderness,<bos_e>,SilverBlaze:Tavistock
4,word.predicate:bet,SilverBlaze:John_Straker,SilverBlaze:Wife_of_John_Straker,SilverBlaze:Silver_Blaze,word.predicate:notHave,SilverBlaze:Night_of_the_incident
5,word.predicate:bring,word.predicate:testify,SilverBlaze:Silver_Blaze,SilverBlaze:Horse,SilverBlaze:wilderness,<bos_e>
6,word.predicate:suspect,word.predicate:collect,SilverBlaze:Night_of_the_incident,SilverBlaze:John_Straker,SilverBlaze:knife,word.predicate:testify
7,word.predicate:train,SilverBlaze:Fitzroy_Simpson,SilverBlaze:Complexion_of_Fitzroy_Simpson,SilverBlaze:In_combat,SilverBlaze:Night_of_the_incident,word.predicate:sleep
8,word.predicate:collect,word.predicate:consider,<bos_e>,word.predicate:wander,word.predicate:testify,SilverBlaze:Fitzroy_Simpson
9,word.predicate:avoid,word.predicate:try,SilverBlaze:Silas_Brown,SilverBlaze:knife,word.predicate:say,word.predicate:notHave


0

['<bos_e>', '<bos_r>', '<bos_e>']

,head,relation,tail,atten_from0
2,<mask_e>,kgc:whom,AbbeyGrange:Sir_Eustace_Brackenstall,0.292831
0,<bos_e>,<bos_r>,<bos_e>,0.216235
5,<mask_e>,kgc:what,<mask_e>,0.138091
4,<mask_e>,kgc:why,<mask_e>,0.114180
6,<mask_e>,kgc:where,<mask_e>,0.113443
3,<mask_e>,kgc:subject,<mask_e>,0.074020
1,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.051201


...

----------


6

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from6
2,<mask_e>,kgc:whom,AbbeyGrange:Sir_Eustace_Brackenstall,0.340453
0,<bos_e>,<bos_r>,<bos_e>,0.307999
5,<mask_e>,kgc:what,<mask_e>,0.084369
6,<mask_e>,kgc:where,<mask_e>,0.082542
4,<mask_e>,kgc:why,<mask_e>,0.067754
3,<mask_e>,kgc:subject,<mask_e>,0.063642
1,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.053242


----------


In [30]:
victim = 'AbbeyGrange:Sir_Eustace_Brackenstall'
df_ranking_AbbeyGrange, df_attension_AbbeyGrange = make_ranking(
    'AbbeyGrange:350', 'AbbeyGrange:413', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_AbbeyGrange.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_AbbeyGrange)
for i in range(len_-20, len_):
    display(i, df_attension_AbbeyGrange.iloc[i,:3].tolist())
    display(df_attension_AbbeyGrange.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:stand,AbbeyGrange:Lady_Brackenstall,AbbeyGrange:Jack_Croker,AbbeyGrange:404,AbbeyGrange:sin,AbbeyGrange:Sydenham
1,word.predicate:notEscape,AbbeyGrange:Sir_Eustace_Brackenstall,AbbeyGrange:police,AbbeyGrange:403,AbbeyGrange:Jack_Croker,AbbeyGrange:bottom_of_pond
2,word.predicate:concider,AbbeyGrange:Burglary_team_of_Lewisham,AllTitle:Holmes,AbbeyGrange:392,AbbeyGrange:truth,ACaseOfIdentity:Holmes_s_office
3,word.predicate:cannotFind,AllTitle:Holmes,AbbeyGrange:Lady_Brackenstall,AbbeyGrange:343,AbbeyGrange:409,ACaseOfIdentity:Between_seams_of_window
4,word.predicate:collect,AbbeyGrange:police,AbbeyGrange:case,AbbeyGrange:Silver_Products,AllTitle:Holmes,ACaseOfIdentity:Pavement_opposite
5,word.predicate:kill,AbbeyGrange:case,AbbeyGrange:Voice_of_the_people,AbbeyGrange:386,AbbeyGrange:appearance,AllTitle:Holmes
6,word.predicate:constrain,AbbeyGrange:Jack_Croker,<bos_e>,AbbeyGrange:375,AbbeyGrange:Sincerity_of_Jack_Croker,AbbeyGrange:Stanley_Hopkins
7,word.predicate:inviteIn,AllTitle:Watson,word.predicate:believe,AbbeyGrange:accomplice,AbbeyGrange:False_charge_person,AbbeyGrange:police
8,word.predicate:canSolve,AbbeyGrange:Stanley_Hopkins,AbbeyGrange:Stanley_Hopkins,AbbeyGrange:police,ACaseOfIdentity:hat,<bos_e>
9,word.predicate:inform,word.predicate:inform,AbbeyGrange:Sir_Eustace_Brackenstall,AbbeyGrange:409,AbbeyGrange:police,AbbeyGrange:Sir_Eustace_Brackenstall


264

['AbbeyGrange:410', 'kgc:hasPredicate', 'word.predicate:say']

,head,relation,tail,atten_from264
277,<bos_e>,<bos_r>,<bos_e>,0.035200
266,<bos_e>,<bos_r>,<bos_e>,0.035114
259,<bos_e>,<bos_r>,<bos_e>,0.035062
262,<bos_e>,<bos_r>,<bos_e>,0.034613
270,<bos_e>,<bos_r>,<bos_e>,0.034092
273,<bos_e>,<bos_r>,<bos_e>,0.030823
254,<bos_e>,<bos_r>,<bos_e>,0.029809
249,<bos_e>,<bos_r>,<bos_e>,0.028509
245,<bos_e>,<bos_r>,<bos_e>,0.021990
267,AbbeyGrange:411,kgc:subject,AbbeyGrange:Jack_Croker,0.020085


----------


...

----------


283

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from283
274,AbbeyGrange:413,kgc:subject,AbbeyGrange:Jack_Croker,0.050374
267,AbbeyGrange:411,kgc:subject,AbbeyGrange:Jack_Croker,0.045212
279,<mask_e>,kgc:whom,AbbeyGrange:Sir_Eustace_Brackenstall,0.042270
276,AbbeyGrange:413,kgc:what,AllTitle:Holmes,0.028692
282,<mask_e>,kgc:what,<mask_e>,0.025086
263,AbbeyGrange:410,kgc:subject,AllTitle:Watson,0.023401
283,<mask_e>,kgc:where,<mask_e>,0.021338
269,AbbeyGrange:411,kgc:what,AbbeyGrange:sin,0.021310
277,<bos_e>,<bos_r>,<bos_e>,0.019092
270,<bos_e>,<bos_r>,<bos_e>,0.018190


----------


### 入院患者
Who killed Blessington? (criminal & explanation)
被害者: Blessington
犯人: 3人
犯行動機: 

In [29]:
victim = 'ResidentPatient:Blessington'
df_ranking_Blessington, df_attension_Blessington = make_ranking(
    'ResidentPatient:250', 'ResidentPatient:323', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_Blessington.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_Blessington)
for i in range(len_-20, len_):
    display(i, df_attension_Blessington.iloc[i,:3].tolist())
    display(df_attension_Blessington.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:kill,ResidentPatient:Blessington,ResidentPatient:Blessington,<bos_e>,ResidentPatient:Source_Date,ResidentPatient:Blessington_room
1,word.predicate:catch,ResidentPatient:Moderator_of_Tobin,ResidentPatient:Elderly_man,SilverBlaze:018a,ResidentPatient:Blessington,ResidentPatient:Blessington
2,word.predicate:leaveBehind,ResidentPatient:Helper_boy,ResidentPatient:Young_man,ResidentPatient:319a,ResidentPatient:The_strength_of_the_fellow_rev...,SilverBlaze:King_s_Pyland
3,word.predicate:escape,ResidentPatient:Percy_Trevelyan,ResidentPatient:Helper_boy,ResidentPatient:Helper_boy,ResidentPatient:324a,SilverBlaze:Dartmoor
4,word.predicate:fail,AbbeyGrange:Lady_Brackenstall,ResidentPatient:Percy_Trevelyan,ResidentPatient:Blessington,ResidentPatient:Own_identity,ResidentPatient:Elderly_man
5,word.predicate:reveal,SilverBlaze:Silver_Blaze,ResidentPatient:319a,word.predicate:free,ResidentPatient:319a,ResidentPatient:Examination_room
6,word.predicate:lock,AllTitle:Holmes,SilverBlaze:Murder_of_John_Straker,ResidentPatient:Young_man,ResidentPatient:decisive_factor,ResidentPatient:Helper_boy
7,word.predicate:betray,ResidentPatient:Young_man,SilverBlaze:People,ResidentPatient:Blessington_room,ResidentPatient:Evidence_of_the_major_events_o...,<bos_e>
8,word.predicate:stand,ResidentPatient:Elderly_man,<bos_e>,ResidentPatient:Elderly_man,<bos_e>,ResidentPatient:Source_Date
9,word.predicate:gain,ACaseOfIdentity:Sutherland,ResidentPatient:Cartwright,ResidentPatient:Percy_Trevelyan,ResidentPatient:death_penalty,ResidentPatient:Among_the_keyhole


375

['ResidentPatient:320', 'kgc:hasPredicate', 'word.predicate:fail']

,head,relation,tail,atten_from375
384,<bos_e>,<bos_r>,<bos_e>,0.035150
380,<bos_e>,<bos_r>,<bos_e>,0.033458
388,<bos_e>,<bos_r>,<bos_e>,0.032763
376,<bos_e>,<bos_r>,<bos_e>,0.031401
373,<bos_e>,<bos_r>,<bos_e>,0.029093
361,<bos_e>,<bos_r>,<bos_e>,0.025119
367,<bos_e>,<bos_r>,<bos_e>,0.025004
336,<bos_e>,<bos_r>,<bos_e>,0.021143
356,<bos_e>,<bos_r>,<bos_e>,0.020062
346,<bos_e>,<bos_r>,<bos_e>,0.018407


----------


...

----------


394

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from394
385,ResidentPatient:323,kgc:subject,ResidentPatient:Blessington,0.031190
393,<mask_e>,kgc:what,<mask_e>,0.025837
369,ResidentPatient:319a,kgc:subject,ResidentPatient:Young_man,0.025742
387,ResidentPatient:323,kgc:what,ResidentPatient:The_strength_of_the_fellow_rev...,0.025504
390,<mask_e>,kgc:whom,ResidentPatient:Blessington,0.025346
379,ResidentPatient:321,kgc:when,ResidentPatient:third_time,0.022435
368,ResidentPatient:319a,kgc:subject,ResidentPatient:Elderly_man,0.021974
381,ResidentPatient:322,kgc:subject,ResidentPatient:Blessington,0.021254
391,<mask_e>,kgc:subject,<mask_e>,0.020154
389,<mask_e>,kgc:hasPredicate,word.predicate:kill,0.019844


----------


### 白銀
Who took out the White Silver Blaze? (criminal & explanation) 
被害者: Silver_Blaze
犯人: 
犯行動機: 

In [27]:
victim = 'SilverBlaze:Silver_Blaze'
df_ranking_SilverBlaze, df_attension_SilverBlaze = make_ranking(
    'SilverBlaze:330', 'SilverBlaze:396', BRING, MASK_E, MASK_E, MASK_E, victim, MASK_E)

display(df_ranking_SilverBlaze.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_SilverBlaze)
for i in range(len_-20, len_):
    display(i, df_attension_SilverBlaze.iloc[i,:3].tolist())
    display(df_attension_SilverBlaze.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:bring,SilverBlaze:Silver_Blaze,AllTitle:Holmes,SilverBlaze:394,SilverBlaze:Silver_Blaze,SilverBlaze:Mapleton_riding_ground_stables
1,word.predicate:give,AllTitle:Holmes,SilverBlaze:Colonel_Ross,SilverBlaze:375,SilverBlaze:Photos_of_John_Straker,SilverBlaze:Wasteland
2,word.predicate:drop,SilverBlaze:Inspector_Gregory,SilverBlaze:John_Straker,SilverBlaze:Silver_Blaze,SilverBlaze:325,SilverBlaze:Dent_of_wasteland
3,word.predicate:get,SilverBlaze:John_Straker,SilverBlaze:Silver_Blaze,SilverBlaze:385,SilverBlaze:337,SilverBlaze:Silver_Blaze
4,word.predicate:notHave,SilverBlaze:Colonel_Ross,SilverBlaze:Inspector_Gregory,SilverBlaze:white_forehead,SilverBlaze:331a,SilverBlaze:wilderness
5,word.predicate:smash,SilverBlaze:Wasteland,SilverBlaze:Youth_stables_number,SilverBlaze:379,SilverBlaze:328,SilverBlaze:John_Straker
6,word.predicate:buy,word.predicate:let,SilverBlaze:3_sheep,SilverBlaze:Brown_Umanoashi_of,SilverBlaze:335,AllTitle:Holmes
7,word.predicate:carry,SilverBlaze:Photos_of_John_Straker,SilverBlaze:Brown_horse,SilverBlaze:387a,SilverBlaze:Work,SilverBlaze:Danger
8,word.predicate:pick,SilverBlaze:394,SilverBlaze:Silas_Brown,SilverBlaze:241,SilverBlaze:How_to_escape_the_sin,<bos_e>
9,word.predicate:fire,word.predicate:ask,SilverBlaze:legs_of_brown_horse,SilverBlaze:377,SilverBlaze:Accident_of_sheep,ACaseOfIdentity:letter_of_Hosma


363

['SilverBlaze:394', 'kgc:subject', 'SilverBlaze:John_Straker']

,head,relation,tail,atten_from363
363,SilverBlaze:394,kgc:subject,SilverBlaze:John_Straker,0.039282
368,SilverBlaze:395,kgc:hasPredicate,word.predicate:have,0.034252
379,<mask_e>,kgc:subject,<mask_e>,0.029990
381,<mask_e>,kgc:what,SilverBlaze:Silver_Blaze,0.026872
365,SilverBlaze:394,kgc:what,SilverBlaze:Expensive_clothes,0.026018
375,SilverBlaze:396,kgc:to,AllTitle:Holmes,0.022384
340,SilverBlaze:390,kgc:hasPredicate,word.predicate:fall,0.021835
352,SilverBlaze:392,kgc:hasPredicate,word.predicate:practice,0.020858
380,<mask_e>,kgc:why,<mask_e>,0.019974
339,SilverBlaze:390,kgc:subject,SilverBlaze:John_Straker,0.019240


----------


...

----------


382

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from382
363,SilverBlaze:394,kgc:subject,SilverBlaze:John_Straker,0.077649
379,<mask_e>,kgc:subject,<mask_e>,0.048823
367,SilverBlaze:395,kgc:subject,SilverBlaze:John_Straker,0.043226
351,SilverBlaze:392,kgc:subject,SilverBlaze:John_Straker,0.037057
381,<mask_e>,kgc:what,SilverBlaze:Silver_Blaze,0.036339
339,SilverBlaze:390,kgc:subject,SilverBlaze:John_Straker,0.028160
375,SilverBlaze:396,kgc:to,AllTitle:Holmes,0.026920
378,<mask_e>,kgc:whom,<mask_e>,0.025507
342,SilverBlaze:390a,kgc:subject,SilverBlaze:John_Straker,0.025033
374,SilverBlaze:396,kgc:what,SilverBlaze:Place_of_the_horse,0.023467


----------


### 花嫁失踪事件（同一事件）
hozmaの失踪の名瀬を探る
被害者: ACaseOfIdentity:Hozma
犯人: 
犯行動機: 

In [34]:
victim = 'ACaseOfIdentity:Hosmer'
df_ranking_ACaseOfIdentity, df_attension_ACaseOfIdentity = make_ranking(
    'ACaseOfIdentity:510', 'ACaseOfIdentity:578', HIDE, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_ACaseOfIdentity.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_ACaseOfIdentity)
for i in range(len_-20, len_):
    display(i, df_attension_ACaseOfIdentity.iloc[i,:3].tolist())
    display(df_attension_ACaseOfIdentity.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")


,predicate,whom,subject,why,what,where
0,word.predicate:know,ACaseOfIdentity:Hosmer,ACaseOfIdentity:Sutherland,ACaseOfIdentity:money,ACaseOfIdentity:Hosmer,ACaseOfIdentity:Company
1,word.predicate:see,ACaseOfIdentity:Sutherland,ACaseOfIdentity:Hosmer,ACaseOfIdentity:Assault_threat,ACaseOfIdentity:Windibank,CrookedMan:Aldershot
2,word.predicate:go,ACaseOfIdentity:Windibank,ACaseOfIdentity:Windibank,ACaseOfIdentity:Illegally,ACaseOfIdentity:handwriting_of_Windibank,ACaseOfIdentity:Church
3,word.predicate:cannotSleep,AllTitle:Holmes,AllTitle:Holmes,<bos_e>,ACaseOfIdentity:Sutherland,ACaseOfIdentity:letter_of_Hosma
4,word.predicate:beTypeed,CrookedMan:Nancy,ACaseOfIdentity:reply,ACaseOfIdentity:Windibank,ACaseOfIdentity:glasses,CrookedMan:main_street
5,word.predicate:believe,AllTitle:Watson,ACaseOfIdentity:feature_of_type,word.predicate:want,ACaseOfIdentity:Moth,<bos_e>
6,word.predicate:have,SilverBlaze:John_Straker,<bos_e>,word.predicate:typist,ACaseOfIdentity:voice,ACaseOfIdentity:France_office_of_company_of_Bo...
7,word.predicate:want,ACaseOfIdentity:mother_of_Sutherland,ACaseOfIdentity:company,word.predicate:canSolve,ACaseOfIdentity:money,word.predicate:go
8,word.predicate:notWant,ACaseOfIdentity:Hardy,AllTitle:Watson,word.predicate:go,ACaseOfIdentity:Color_glasses,ACaseOfIdentity:Dance_party
9,word.predicate:happen,ACaseOfIdentity:Personal_statement,ACaseOfIdentity:Personal_statement,word.predicate:notAppear,ACaseOfIdentity:letter,AbbeyGrange:Sydenham


303

['ACaseOfIdentity:575', 'kgc:what', 'ACaseOfIdentity:Hosmer']

,head,relation,tail,atten_from303
302,ACaseOfIdentity:575,kgc:hasPredicate,word.predicate:match,0.116927
301,ACaseOfIdentity:575,kgc:subject,ACaseOfIdentity:feature_of_type,0.041648
303,ACaseOfIdentity:575,kgc:what,ACaseOfIdentity:Hosmer,0.034078
305,ACaseOfIdentity:576,kgc:subject,ACaseOfIdentity:company,0.024203
319,<mask_e>,kgc:subject,<mask_e>,0.023159
282,ACaseOfIdentity:570,kgc:from,ACaseOfIdentity:Personal_statement,0.023116
309,ACaseOfIdentity:577,kgc:subject,ACaseOfIdentity:Personal_statement,0.022063
317,<mask_e>,kgc:hasPredicate,word.predicate:hide,0.020441
322,<mask_e>,kgc:where,<mask_e>,0.019928
281,ACaseOfIdentity:570,kgc:what,ACaseOfIdentity:voice,0.017806


----------


...

----------


322

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from322
301,ACaseOfIdentity:575,kgc:subject,ACaseOfIdentity:feature_of_type,0.056671
305,ACaseOfIdentity:576,kgc:subject,ACaseOfIdentity:company,0.055875
319,<mask_e>,kgc:subject,<mask_e>,0.055015
318,<mask_e>,kgc:whom,ACaseOfIdentity:Hosmer,0.054688
313,ACaseOfIdentity:578,kgc:subject,ACaseOfIdentity:Sutherland,0.052709
298,ACaseOfIdentity:574,kgc:subject,ACaseOfIdentity:reply,0.046229
309,ACaseOfIdentity:577,kgc:subject,ACaseOfIdentity:Personal_statement,0.039633
321,<mask_e>,kgc:what,<mask_e>,0.038679
322,<mask_e>,kgc:where,<mask_e>,0.037073
317,<mask_e>,kgc:hasPredicate,word.predicate:hide,0.033318


----------
